<a href="https://colab.research.google.com/github/YugaYoshimura/a/blob/main/sotsuron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Chromiumとseleniumをインストール
print("前処理を開始")
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

前処理を開始
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,237 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 htt

In [ ]:
import time
import json
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
webdriver.__version__

'4.7.2'

In [ ]:
pd.options.display.max_rows = 200

# アカウント情報の読込
account = pd.read_csv("account.txt", header=None, sep=',')
MAIL = account.loc[0,0]
PASS = account.loc[0,1]

In [ ]:
def login_fun(_mail, _pass):
    # ログインページにアクセス
    driver.get("https://m-league.jp/supporter/login")
    
    # Mail, Passを入力
    id_input = driver.find_element_by_id("login_mail")
    id_input.send_keys(_mail)
    pass_input = driver.find_element_by_id("login_password")
    pass_input.send_keys(_pass)
    
    # ログインボタンをクリック
    login_button = driver.find_elements_by_name("login")
    login_button[1].click()

def get_match_schedule_fun(season):
    
    match_schedule_list = []
    
    # 試合日程ページへアクセス
    if season == 2020:
        url = "https://m-league.jp/games"
    else:
        url = f"https://m-league.jp/games/{season}-season"
    driver.get(url)
    
    # 20xxシーズンは20xx+1まで開催
    for y in [season, season+1]:
        for m in range(1, 12+1, 1):
                for d in range(1, 31+1, 1):
                    ymd = str(y) + str(m).zfill(2) + str(d).zfill(2)
                    if len(driver.find_elements_by_xpath(f".//div[@id='js-modal-key{ymd}']")) == 1:
                        match_schedule_list += [ymd]
    
    return match_schedule_list

def make_score_data_fun(html):
    # htmlをパースする
    soup = BeautifulSoup(html, "html.parser")
    
    # 必要な個所だけ抽出
    start_mark = "UMP_PLAYER.init(true, true, \'"
    end_mark = "\', autoplay);"
    start_position = str(soup).find(start_mark) + len(start_mark)
    end_position = str(soup).find(end_mark)
    score = str(soup)[start_position:end_position]
    
    # DataFrameに変換
    dat = pd.DataFrame(json.loads(score))
    
    return dat

def make_dat_2020_fun(match_schedule_list):
    driver.get("https://m-league.jp/games")

    dat_list = []
    button_count = 0
    # 試合日程は時系列にソート済み想定
    for m_s in match_schedule_list:
        # 月選択
        tmp = driver.find_element_by_xpath(f".//a[@href='/games/?mly={m_s[0:4]}&mlm={int(m_s[4:6])}#schedule']")
        tmp.click()
        # 日程選択
        tmp = driver.find_element_by_xpath(f".//li[@data-target='key{m_s}']")
        tmp.click()
        
        time.sleep(1)
        # 牌譜ビューアにアクセスするボタン取得
        button = driver.find_elements_by_xpath(".//button[@class='c-button -primary -small']")
        button[button_count].click()

        ### 第1試合
        # 制御するタブの移動
        driver.switch_to.window(driver.window_handles[1])
        # 文字コードをUTF-8に変換
        html = driver.page_source.encode('utf-8')
        # DataFrameに変換
        dat = make_score_data_fun(html)
        dat_list += [dat]
        # 牌譜ビューアを閉じる
        driver.close()
        # 制御するタブの移動
        driver.switch_to.window(driver.window_handles[0])

        ### 第2試合
        button[button_count+1].click()
        # 制御するタブの移動
        driver.switch_to.window(driver.window_handles[1])
        # 文字コードをUTF-8に変換
        html = driver.page_source.encode('utf-8')
        # DataFrameに変換
        dat = make_score_data_fun(html)
        dat_list += [dat]
        # 牌譜ビューアを閉じる
        driver.close()
        # 制御するタブの移動
        driver.switch_to.window(driver.window_handles[0])

        # ページのリセット
        driver.get("https://m-league.jp/games")
        # button_countを次の日程にズラす
        button_count += 2

        # アクセス制限
        time.sleep(10)
        print("Get: " + m_s)
        
    # データ縦積み
    dat_all = pd.concat(dat_list).reset_index(drop=True)
    
    return dat_all

In [ ]:
# ブラウザのオプション
options = Options()

# Headless
options.headless = False

#サンドボックスモードの解除。これも必須。
options.add_argument('--no-sandbox')

# ブラウザ起動
driver = webdriver.Chrome(executable_path="/content/chromedriver.exe", options=options)

<ipython-input-31-7d2ca904506b>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/content/chromedriver.exe", options=options)


WebDriverException: ignored